In [ ]:
from langchain.chat_models import init_chat_model
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import interrupt, Command
from dotenv import load_dotenv
load_dotenv()

In [ ]:
class MessageState(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
import httpx
from typing import Any
from langchain_core.tools import tool
import asyncio

API_BASE = "https://api.weather.gov"
user_agent = "weather-app/1.0"

async def make_request(url: str) -> dict[str, Any] | None:
    """Make a request to the NWS API with proper error handling."""
    headers = {
        "User-Agent": user_agent,
        "Accept": "application/geo+json"
    }

    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(url, headers=headers, timeout=30.0)
            response.raise_for_status()
            return response.json()
        except Exception:
            return None

def feature_alert(feature: str) -> str:
    feat = feature["properties"]
    return f"""
    Event: {feat.get('event', 'unknown')}
    Area: {feat.get('areaDesc', 'Unknown')}
    Severity: {feat.get('severity', 'Unknown')}
    Description: {feat.get('description', 'No description available')}
    Instructions: {feat.get('instruction', 'No specific instructions provided')}
    """

@tool
def get_alerts(state: str) -> str:
    """Get weather alerts for a US state.

    Args:
        state: Two-letter US state code (e.g. CA, NY)
    """
    import nest_asyncio
    nest_asyncio.apply()
    
    url = f"{API_BASE}/alerts/active/area/{state}"
    
    # Try to get or create event loop
    try:
        loop = asyncio.get_event_loop()
        if loop.is_closed():
            raise RuntimeError("Event loop is closed")
    except RuntimeError:
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
    
    data = loop.run_until_complete(make_request(url))

    if not data or "features" not in data:
        return "Unable to fetch alerts"

    if not data["features"]:
        return "No active alerts"
    # decision = interrupt("Approve to proceed to use weather tool")
    # if decision=="yes":
    #     alerts = [feature_alert(feature) for feature in data["features"]]
    #     return "\n---\n".join(alerts)
    # else:
    #     return "Error"
    alerts = [feature_alert(feature) for feature in data["features"]]
    return "\n---\n".join(alerts)

tools = [get_alerts]
llm = init_chat_model("google_genai:gemini-2.0-flash")
llm_with_tools = llm.bind_tools(tools)

In [ ]:
def chatbot(state: MessageState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# memory=MemorySaver()
builder=StateGraph(MessageState)

builder.add_node("chatbot",chatbot)
builder.add_node("tools",ToolNode(tools))

builder.add_edge(START, "chatbot")
builder.add_conditional_edges("chatbot", tools_condition)
builder.add_edge("tools","chatbot")
builder.add_edge("chatbot", END)

graph=builder.compile()
# graph=builder.compile(checkpointer=memory)
# config = {"configurable": {"thread_id": "buy_thread"}}

In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
state = graph.invoke({"messages": [{"role": "user", "content": "Get weather alerts for California CA"}]})
# print(state.get("__interrupt__"))  # Shows the interrupt prompt
# decision = input("Approve (yes/no): ")
# state = graph.invoke(Command(resume=decision), config=config)
print(state["messages"][-1].content)

In [ ]:
from langsmith import traceable

@traceable
def call_graph(query: str):
    state = graph.invoke({"messages": [{"role": "user", "content": query}]})
    return state["messages"][-1].content

call_graph("what is the weather in Arizona AZ")